In [2]:
from theano.sandbox import cuda
cuda.use('gpu1')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5103)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [3]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [4]:
path = get_file('nietzche.txt', 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read()
print("Corpus length: %s" % len(text))

Corpus length: 600901


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars) + 1
chars.insert(0, "\0")

In [6]:
print("Total chars: ", vocab_size)

Total chars:  86


In [7]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz'

In [8]:
char_indices = dict((c, i) for (i, c) in enumerate(chars))
indices_char = dict((i, c) for (i, c) in enumerate(chars))

In [9]:
idx = [char_indices[c] for c in text]

In [10]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [11]:
''.join([chars[i] for i in idx[:100]])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all ph'

# 3-Char Model

In [12]:
seq_length = 3
char_0 = [idx[i] for i in range(0, len(idx)-1-seq_length, seq_length)]
char_1 = [idx[i + 1] for i in range(0, len(idx)-1-seq_length, seq_length)]
char_2 = [idx[i + 2] for i in range(0, len(idx)-1-seq_length, seq_length)]
char_3 = [idx[i + 3] for i in range(0, len(idx)-1-seq_length, seq_length)]

In [111]:
x0 = np.array(char_0[:-2])
x1 = np.array(char_1[:-2])
x2 = np.array(char_2[:-2])
x3 = np.array(char_3[:-2])

In [19]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [113]:
n_fac = 42

In [159]:
inp0, emb0 = embedding_input('c0_dat', vocab_size, n_fac)
inp1, emb1 = embedding_input('c1_dat', vocab_size, n_fac)
inp2, emb2 = embedding_input('c2_dat', vocab_size, n_fac)

In [160]:
n_hidden = 256

In [161]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='tanh')
dense_out = Dense(vocab_size, activation='softmax')

In [162]:
# these embs are already linked to the inps from the embedding_input function
hidden = dense_in(emb0)
c2_dense = dense_in(emb1)
hidden = merge([dense_hidden(hidden), c2_dense])
c3_dense = dense_in(emb2)
hidden = merge([dense_hidden(hidden), c3_dense])
c4_out = dense_out(hidden)

In [163]:
model = Model([inp0, inp1, inp2], c4_out)

In [164]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
# model.optimizer.lr = 0.000001
model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 8s - loss: 2.4012     
Epoch 2/4
200297/200297 [==============================] - 8s - loss: 2.2555     
Epoch 3/4
200297/200297 [==============================] - 7s - loss: 2.2093     
Epoch 4/4
200297/200297 [==============================] - 7s - loss: 2.1703     


In [138]:
model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 7s - loss: 3.1080     
Epoch 2/4
200297/200297 [==============================] - 8s - loss: 3.0952     
Epoch 3/4
200297/200297 [==============================] - 7s - loss: 3.0848     
Epoch 4/4
200297/200297 [==============================] - 7s - loss: 3.0757     


In [139]:
model.optimizer.lr = 0.01

In [140]:
model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 7s - loss: 3.0675     
Epoch 2/4
200297/200297 [==============================] - 7s - loss: 3.0595     
Epoch 3/4
200297/200297 [==============================] - 7s - loss: 3.0518     
Epoch 4/4
200297/200297 [==============================] - 7s - loss: 3.0440     


In [141]:
model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 7s - loss: 3.0361     
Epoch 2/4
200297/200297 [==============================] - 7s - loss: 3.0281     
Epoch 3/4
200297/200297 [==============================] - 7s - loss: 3.0200     
Epoch 4/4
200297/200297 [==============================] - 7s - loss: 3.0117     


In [142]:
model.optimizer.lr = 0.000001

In [143]:
model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 7s - loss: 3.0034     
Epoch 2/4
200297/200297 [==============================] - 7s - loss: 2.9950     
Epoch 3/4
200297/200297 [==============================] - 7s - loss: 2.9866     
Epoch 4/4
200297/200297 [==============================] - 7s - loss: 2.9783     


In [170]:
model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 8s - loss: 2.1393     
Epoch 2/4
200297/200297 [==============================] - 8s - loss: 2.1175     
Epoch 3/4
200297/200297 [==============================] - 8s - loss: 2.1019     
Epoch 4/4
200297/200297 [==============================] - 8s - loss: 2.0905     


In [171]:
model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 8s - loss: 2.0824     
Epoch 2/4
200297/200297 [==============================] - 8s - loss: 2.0748     
Epoch 3/4
200297/200297 [==============================] - 8s - loss: 2.0689     
Epoch 4/4
200297/200297 [==============================] - 8s - loss: 2.0647     


In [172]:
model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 8s - loss: 2.0598     
Epoch 2/4
200297/200297 [==============================] - 8s - loss: 2.0565     
Epoch 3/4
200297/200297 [==============================] - 8s - loss: 2.0534     
Epoch 4/4
200297/200297 [==============================] - 8s - loss: 2.0504     


In [173]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [174]:
get_next('the')

' '

In [175]:
get_next('thi')

't'

In [176]:
get_next('hel')

' '

In [177]:
get_next('Som')

'i'

In [178]:
get_next('Wha')

't'

In [179]:
get_next('is ')

't'

In [180]:
get_next('s t')

'h'

# Our First RNN